In [1]:
from Parser import parse_frames
from scipy.signal import medfilt
from Functions import find_extremas, filter_extremas, detect_side, analyse_each_rep
from JointAngles import JointAngles
import numpy as np
import math
import glob
import os
import itertools
import itertools
import pickle
import matplotlib.pyplot as plt


In [68]:
np.delete(np.array([1,2,3,4,5,6,6]), 4)

array([1, 2, 3, 4, 6, 6])

In [84]:
for a, b in itertools.combinations([4,3,2,1], 2):
    print(a,b)
        
  
    



4 3
4 2
4 1
3 2
3 1
2 1


In [75]:
np.arange(44, 59)

array([44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58])

In [2]:
# modify this path for other videos 
# input_folder = "/home/aleko/Documents/Projects/IndividualProject/bicepvid"
# input_folder = "C:\\Users\\altaok\\Documents\\GitHub\\IndividualProject\\keypoints_for_all\\shoulder_press"
input_folder = 'C:\\Users\\ak5u16\\Desktop\\openpose\\keypoints_for_all\\shoulder press'
folder_paths = glob.glob(os.path.join(input_folder, 'shoulder_press_correct*'))     

        

for folder in folder_paths:
    print("Starting..." + "\nKeypoints from folder: " + os.path.basename(folder))
    video_name = os.path.basename(folder).replace('output_points_', '')
    frame_poses = parse_frames(folder)
       
    joint_angles = JointAngles('shoulder press', frame_poses)
    print('Detected side: ' + joint_angles.side)
   

    left_upArm_trunk_angles = np.array(joint_angles.left_upArm_trunk_angles)
    left_upArm_trunk_angles_filtered = medfilt(medfilt(left_upArm_trunk_angles, 5), 5)
    
    right_upArm_trunk_angles = np.array(joint_angles.right_upArm_trunk_angles)
    right_upArm_trunk_angles_filtered = medfilt(medfilt(right_upArm_trunk_angles, 5), 5)
    
    left_upArm_forearm_angles = np.array(joint_angles.left_upArm_forearm_angles)
    left_upArm_forearm_angles_filtered = medfilt(medfilt(left_upArm_forearm_angles, 5), 5)
    
    right_upArm_forearm_angles = np.array(joint_angles.right_upArm_forearm_angles)
    right_upArm_forearm_angles_filtered = medfilt(medfilt(right_upArm_forearm_angles, 5), 5)
    
        
    # Find left upper arm and trunk maximum angles to detect wrong form 
    left_upArm_trunk_maximas, indexesmax1a = find_extremas(left_upArm_trunk_angles_filtered)
    left_upArm_trunk_maximas = filter_extremas(left_upArm_trunk_angles_filtered, left_upArm_trunk_maximas)
    
    # print("Largest angles between upper arm and trunk: " + str(upArm_trunk_maximas))
    # Find right upper arm and trunk maximum angles to detect wrong form 
    right_upArm_trunk_maximas, indexesmax1b = find_extremas(right_upArm_trunk_angles_filtered)
    right_upArm_trunk_maximas = filter_extremas(right_upArm_trunk_angles_filtered, right_upArm_trunk_maximas)
    
    # Count repetitions
    left_upArm_forearm_maximas, indexesmax2a = find_extremas(left_upArm_forearm_angles_filtered)    
    left_upArm_forearm_maximas = filter_extremas(left_upArm_forearm_angles_filtered, left_upArm_forearm_maximas)
    # print("Number of maxima points between Upper Arm and Forearm: " + str(upArm_forearm_maximas.size))
    # print(upArm_forearm_maximas)
    
    right_upArm_forearm_maximas, indexesmax2b = find_extremas(right_upArm_forearm_angles_filtered) 
    right_upArm_forearm_maximas = filter_extremas(right_upArm_forearm_angles_filtered, right_upArm_forearm_maximas)
   

    # Find upper arm and forearm minimum points to count reps     
    left_upArm_forearm_minimas, indexesmin1a = find_extremas(left_upArm_forearm_angles_filtered, maxima=False)
    print('----------------------\nFiltering minimas')
    print(left_upArm_forearm_minimas)
    left_upArm_forearm_minimas = filter_extremas(left_upArm_forearm_angles_filtered, left_upArm_forearm_minimas)
  
    print("Number of reps performed LEFT: " + str(left_upArm_forearm_minimas.size))
    # print(left_upArm_forearm_minimas)
    
    right_upArm_forearm_minimas, indexesmin1b = find_extremas(right_upArm_forearm_angles_filtered, maxima=False)
    print('----------------------\nFiltering minimas')
    print(right_upArm_forearm_minimas)
    right_upArm_forearm_minimas = filter_extremas(right_upArm_forearm_angles_filtered, right_upArm_forearm_minimas)
    print("Number of reps performed RIGHT: " + str(right_upArm_forearm_minimas.size))
    
    left_upArm_trunk_minimas, indexesmin2a = find_extremas(left_upArm_trunk_angles_filtered, maxima=False)
    right_upArm_trunk_minimas, indexesmin2b = find_extremas(right_upArm_trunk_angles_filtered, maxima=False)

    print('\n'+ '-'*120 + '\nLEFT ARM'.center(100) + '\n' + '-'*120 )
    # Plot 1 - Left
    plt.figure(figsize=(10,4))
    plt.scatter(np.arange(left_upArm_trunk_angles.shape[0]), left_upArm_trunk_angles, alpha=0.5)
    plt.scatter(np.arange(left_upArm_trunk_angles_filtered.shape[0]), left_upArm_trunk_angles_filtered, c='r', alpha=0.5)
    plt.title(video_name)
    plt.xlabel('Frames')
    plt.ylabel('Angle between LEFT Upper Arm and Trunk')
    # Set range on y-axis so the plots are consistent
    plt.ylim(0,200)
    
    # Annontate minima points
    plt.plot(indexesmin2a, left_upArm_trunk_angles_filtered[indexesmin2a], 'bo')
    # print(indexesmax1a)
    plt.show()
    
    # Plot 2 - Left
    plt.figure(figsize=(10,4))
    plt.scatter(np.arange(left_upArm_forearm_angles.shape[0]), left_upArm_forearm_angles, alpha=0.5)
    plt.scatter(np.arange(left_upArm_forearm_angles_filtered.shape[0]), left_upArm_forearm_angles_filtered, c='r', alpha=0.5)
    plt.title(video_name)
    plt.xlabel('Frames')
    plt.ylabel('Angle between LEFT Upper Arm and Forearm')
    # Set range on y-axis so the plots are consistent
    plt.ylim(0,200) 
    
    # Annotate minima apoints on the graph   
    max_points1a = left_upArm_forearm_angles_filtered[indexesmax2a]
    min_points1a = left_upArm_forearm_angles_filtered[indexesmin1a]
    plt.plot(indexesmin1a, min_points1a, 'bo')
    plt.plot(indexesmax2a, max_points1a, 'ks')

    """
        # Annotate local maximas
        for maxp, indx in zip(max_points, indexesmax2):
            plt.annotate('local max', xy=(indx, maxp), xytext=(indx, maxp+25),
                    arrowprops=dict(facecolor='black', shrink=0.05))

        # Annotate local minimas
        for minp, indx in zip(min_points, indexesmin):
            plt.annotate('local min', xy=(indx, minp), xytext=(indx, minp-35),
                    arrowprops=dict(facecolor='black', shrink=0.05))     
    """
    
    plt.show()
    
    # Plot 3 - All angles LEFT
    plt.figure(figsize=(10,4))
    plt.scatter(np.arange(left_upArm_forearm_angles_filtered.shape[0]), left_upArm_forearm_angles_filtered, c='r', alpha=0.5)
    plt.scatter(np.arange(left_upArm_trunk_angles_filtered.shape[0]), left_upArm_trunk_angles_filtered, c='r', alpha=0.5)
    
    plt.title(video_name)
    plt.xlabel('Frames')
    plt.ylabel('Whole body movement LEFT')
    # Set range on y-axis so the plots are consistent
    plt.ylim(0,200) 
    # Annotate minima apoints   
    plt.plot(indexesmin1a, left_upArm_forearm_angles_filtered[indexesmin1a], 'bo')
    plt.plot(indexesmax1a, left_upArm_trunk_angles_filtered[indexesmax1a], 'ks')
    plt.plot(indexesmax2a, left_upArm_forearm_angles_filtered[indexesmax2a], 'ks')  
    plt.show()
   
    
    #---------------------------------------------------------------------------------------------------------------------
    # Same for right side
    
    print('\n'+ '-'*120 + '\nRIGHT ARM'.center(200) + '\n' + '-'*120 )
    
    # Plot 3 - Right
    plt.figure(figsize=(10,4))
    plt.scatter(np.arange(right_upArm_trunk_angles.shape[0]), right_upArm_trunk_angles, alpha=0.5)
    plt.scatter(np.arange(right_upArm_trunk_angles_filtered.shape[0]), right_upArm_trunk_angles_filtered, c='r', alpha=0.5)
    plt.title(video_name)
    plt.xlabel('Frames')
    plt.ylabel('Angle between RIGHT Upper Arm and Trunk')
    # Set range on y-axis so the plots are consistent
    plt.ylim(0,200)
    
    # Annontate minima points
    plt.plot(indexesmin2b, right_upArm_trunk_angles_filtered[indexesmin2b], 'bo')
    plt.show()

    # Plot 3 - Right
    plt.figure(figsize=(10,4))
    plt.scatter(np.arange(right_upArm_forearm_angles.shape[0]), right_upArm_forearm_angles, alpha=0.5)
    plt.scatter(np.arange(right_upArm_forearm_angles_filtered.shape[0]), right_upArm_forearm_angles_filtered, c='r', alpha=0.5)
    plt.title(video_name)
    plt.xlabel('Frames')
    plt.ylabel('Angle between RIGHT Upper Arm and Forearm')
    # Set range on y-axis so the plots are consistent
    plt.ylim(0,200) 
    
    # Annotate minima apoints on the graph   
    max_points1b = right_upArm_forearm_angles_filtered[indexesmax2b]
    min_points1b = right_upArm_forearm_angles_filtered[indexesmin1b]
    plt.plot(indexesmin1b, min_points1b, 'bo')
    plt.plot(indexesmax2b, max_points1b, 'ks')
    
    """
        # Annotate local maximas
        for maxp, indx in zip(max_points1b, indexesmax2b):
            plt.annotate('local max', xy=(indx, maxp), xytext=(indx, maxp+25),
                    arrowprops=dict(facecolor='black', shrink=0.05))

        # Annotate local minimas
        for minp, indx in zip(min_points, indexesmin):
            plt.annotate('local min', xy=(indx, minp), xytext=(indx, minp-35),
                    arrowprops=dict(facecolor='black', shrink=0.05))     
    """
    plt.show()


    # Plot 4 - All angles RIGHT
    plt.figure(figsize=(10,4))
    plt.scatter(np.arange(right_upArm_forearm_angles_filtered.shape[0]), right_upArm_forearm_angles_filtered, c='r', alpha=0.5)
    plt.scatter(np.arange(right_upArm_trunk_angles_filtered.shape[0]), right_upArm_trunk_angles_filtered, c='r', alpha=0.5)
    plt.title(video_name)
    plt.xlabel('Frames')
    plt.ylabel('Whole body movement RIGHT')
    # Set range on y-axis so the plots are consistent
    plt.ylim(0,200) 
    # Annotate minima apoints     
    plt.plot(indexesmin1b, right_upArm_forearm_angles_filtered[indexesmin1b], 'bo')
    plt.plot(indexesmax1b, right_upArm_trunk_angles_filtered[indexesmax1b], 'ks')
    plt.plot(indexesmax2b, right_upArm_forearm_angles_filtered[indexesmax2b], 'ks')
    plt.show()
   
    
    # Each rep analysis
    # local minimums for upper arm and forearm are min angles in each rep
    analyse_each_rep(string='analysis', extremas1=left_upArm_forearm_minimas, uf_angles1a=left_upArm_forearm_angles_filtered, ut_angles2a=left_upArm_trunk_angles_filtered,
                    extremas2=right_upArm_forearm_minimas, uf_angles1b=right_upArm_forearm_angles_filtered, ut_angles2b=right_upArm_trunk_angles_filtered)
    

    print('\n' + '='*120 + '\n')


Starting...
Keypoints from folder: shoulder_press_correct_1
Detected side: front
first extrema index: 23
[62, 63, 66, 61, 65]
average count: 63
6
0 63
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62]
size after: 6
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30

 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 

 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 

0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 1

 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 

...

 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 

 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 

 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 

 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126]
0 127
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
a = np.array([1,2,3])
print(a[1:])

[2 3]


In [4]:
np.arange(24)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [4]:
for a, b in itertools.combinations([1,2,3,4,4], 2):
    print(a+b)

3
4
5
5
5
6
6
7
7
8


In [14]:
a = np.array([1,3,4,5])
index = np.where(a == 3)
print(index[0])
a = np.delete(a, index[0][0])
print(a)


[1]
[1 4 5]


In [15]:
d = {1:'a', 2:'b'}
g = {1:'b', 2:'t'}

ds = [d, g]
a = {}
for k in d.keys():
    a[k] = tuple(d[k] for d in ds)
print(a.items())

dict_items([(1, ('a', 'b')), (2, ('b', 't'))])
